In [1]:
%load_ext autoreload
%autoreload 2

import cfr
import pandas as pd
import LMRt

In [2]:
job = cfr.ReconJob(verbose=True)
job.load_proxydb('./data/pages2k_updated_Palmyra_dataset.pkl', verbose=True)

>>> job.configs:
{}
>>> job.configs["proxydb_path"] = ./data/pages2k_updated_Palmyra_dataset.pkl
>>> 692 records loaded
>>> job.proxydb created


In [3]:
pids = [
    'Ocn_065',
    'Ocn_075',
    'Ocn_096',
    'Ocn_101',
    'Ocn_070',
    'Ocn_103',
    'Ocn_077',
    'Ocn_122',
    'Ocn_095',
    'Ocn_104',
    'Ocn_125',
    'Ocn_087',
]

job.filter_proxydb(by='pid', keys=pids)

# pobj = job.proxydb.records['Ocn_075']
pobj = job.proxydb.records['Ocn_103']
pobj.time

array([ 928.125,  928.209,  928.292, ..., 1998.21 , 1998.294, 1998.377])

In [4]:
import xarray as xr

dates = cfr.utils.year_float2datetime(pobj.time)
da_wk = xr.DataArray(
    pobj.value, dims=['time'], coords={'time': dates}
)
da_wk

<xarray.DataArray (time: 6384)>
array([-4.651, -4.631, -4.629, ..., -5.469, -5.413, -5.262])
Coordinates:
  * time     (time) object 0928-02-01 00:00:00 ... 1998-05-01 00:00:00

In [5]:
months = [12, 1, 2]
sda = da_wk.sel(time=da_wk.time.dt.month.isin([months]))
sda

<xarray.DataArray (time: 1596)>
array([-4.651, -4.546, -4.471, ..., -5.48 , -5.435, -5.493])
Coordinates:
  * time     (time) object 0928-02-01 00:00:00 ... 1998-02-01 00:00:00

In [8]:
anchor = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN', 'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']
idx = months[-1]-1
print(anchor[idx])
sda_ann = sda.resample(time=f'A-{anchor[idx]}').mean()
sda_ann

FEB


<xarray.DataArray (time: 1071)>
array([-4.651     , -4.48166667, -4.54933333, ..., -5.009     ,
       -5.20033333, -5.46933333])
Coordinates:
  * time     (time) object 0928-02-29 00:00:00 ... 1998-02-28 00:00:00

In [7]:
import numpy as np
time_ann = np.floor(cfr.utils.datetime2year_float(sda_ann.time.values))
print(time_ann)
print(len(time_ann))

[ 928.  929.  930. ... 1996. 1997. 1998.]
1071


In [8]:
job.annualize_proxydb(months=[12, 1, 2], ptypes=['coral'], verbose=True)

>>> job.configs["annualize_proxydb_months"] = [12, 1, 2]
>>> job.configs["annualize_proxydb_ptypes"] = ['coral']


Annualizing ProxyRecord: 100%|██████████| 12/12 [00:00<00:00, 28.34it/s]

>>> 12 records remaining
>>> job.proxydb updated


In [9]:
import numpy as np

job_old = pd.read_pickle('./data/job.pkl')
for pid, pobj in job.proxydb.records.items():
    pobj_old = job_old.proxydb.records[pid]
    try:
        print(
            pobj.time[0], pobj_old.time[0],
            pobj.time[-1], pobj_old.time[-1],
        )
        print(np.sum(pobj.value - pobj_old.value))
    except:
        print(pid)

1551.0 1552 1990.0 1991
0.0
1659.0 1659 1995.0 1994
Ocn_075
1726.0 1726 1996.0 1996
-0.12499999999999467
1648.0 1648 1999.0 1998
Ocn_101
1733.0 1733 2008.0 2007
Ocn_070
928.0 928 1998.0 1997
Ocn_103
1621.0 1622 1998.0 1997
Ocn_077
1637.0 1638 1982.0 1983
0.0
1616.0 1617 2000.0 2001
0.0
1707.0 1707 1984.0 1983
Ocn_104
1726.0 1726 1996.0 1995
Ocn_125
1606.0 1607 1980.0 1981
0.0


In [10]:
pid = 'Ocn_103'
pobj_old = job_old.proxydb.records[pid]
pobj = job.proxydb.records[pid]

for t in pobj.time:
    idx_old = list(pobj_old.time).index(t)
    idx = list(pobj.time).index(t)
    print(pobj_old.value[idx_old], pobj.value[idx])

-4.4816666666666665 -4.556
-4.549333333333333 -4.512666666666666
-4.7059999999999995 -4.656
-4.634666666666667 -4.662333333333333
-4.6930000000000005 -4.6803333333333335
-4.5249999999999995 -4.599
-4.475666666666666 -4.488
-4.361666666666667 -4.364333333333334
-4.407666666666667 -4.401000000000001
-4.595666666666667 -4.532333333333334
-4.370666666666667 -4.447333333333333
-4.246666666666666 -4.304666666666667
-4.265666666666667 -4.233666666666666
-4.365333333333333 -4.356333333333333
-4.435666666666667 -4.4206666666666665
-4.312333333333333 -4.3549999999999995
-4.164666666666666 -4.175666666666667
-4.450666666666667 -4.399
-4.459 -4.418
-4.372333333333334 -4.431
-4.627 -4.533333333333334
-4.522666666666667 -4.5409999999999995
-4.461333333333333 -4.476333333333334
-4.388999999999999 -4.419666666666667
-4.383333333333333 -4.383333333333334
-4.516666666666667 -4.492999999999999
-4.690666666666666 -4.6419999999999995
-4.425 -4.488333333333333
-4.6129999999999995 -4.558
-4.483333333333333 -

ValueError: 1464.0 is not in list

In [ ]:
pid = 'Ocn_077'
pobj_old = job_old.proxydb.records[pid]
pobj = job.proxydb.records[pid]

for t in pobj.time:
    idx_old = list(pobj_old.time).index(t)
    idx = list(pobj.time).index(t)
    print(pobj_old.value[idx_old], pobj.value[idx])

-3.852 -4.055
-4.145 -3.852
-3.77 -4.145
-3.975 -3.77
-4.069 -3.975
-4.045 -4.069
-4.104 -4.045
-3.977 -4.104
-3.944 -3.977
-3.926 -3.944
-4.05 -3.926
-4.015 -4.05
-4.157 -4.015
-3.983 -4.157
-3.952 -3.983
-4.208 -3.952
-4.096 -4.208
-4.049 -4.096
-4.097 -4.049
-4.189 -4.097
-4.143 -4.189
-4.088 -4.143
-4.054 -4.088
-4.079 -4.054
-4.356 -4.079
-3.995 -4.356
-3.836 -3.995
-4.218 -3.836
-4.03 -4.218
-4.049 -4.03
-4.08 -4.049
-4.103 -4.08
-4.043 -4.103
-4.178 -4.043
-4.076 -4.178
-4.221 -4.076
-4.342 -4.221
-4.325 -4.342
-4.133 -4.325
-4.124 -4.133
-4.339 -4.124
-4.236 -4.339
-4.017 -4.236
-4.275 -4.017
-3.908 -4.275
-4.039 -3.908
-4.13 -4.039
-3.923 -4.13
-3.981 -3.923
-4.009 -3.981
-4.067 -4.009
-3.967 -4.067
-3.923 -3.967
-3.804 -3.923
-3.948 -3.804
-4.175 -3.948
-4.008 -4.175
-4.118 -4.008
-4.032 -4.118
-4.271 -4.032
-3.976 -4.271
-4.079 -3.976
-4.201 -4.079
-3.818 -4.201
-4.166 -3.818
-4.212 -4.166
-4.254 -4.212
-4.034 -4.254
-4.002 -4.034
-4.108 -4.002
-4.264 -4.108
-4.149 -4.264
-4

ValueError: 1722.0 is not in list